In [5]:
o_corn_crop

[258, 399, 1505, 1646]

In [1]:
import base
import numpy as np
import pandas as pd
import itertools
import network
import igraph as ig

class SelectiveNetwork(network.ResistiveNetwork):
    def __init__(self, directory):
        super().__init__(directory)

    def effectiveR(self):
        Q = np.linalg.pinv(self.L)  
        self.O_eff = Q[-1,-1]+Q[-2,-2]-2*Q[-1,-2]
        return self.O_eff

In [1]:
#Aligned analysis - object implementation; discrete rotations
h = SelectiveNetwork('TestData/AgNWN_10um')
h.stack_to_gsd(crop=[0,1000,0,1000])
h.G_u()
h.potential_distribution(0, [0,10], [990,1000], rho_dim=0.89, R_j=7)
print(h.effectiveR())

#base.Node_labelling(g, g.P, 'P', 'orth.gsd')

#df.to_csv('AgNWN.csv')

Ran stack_to_gsd() in  0.3453350067138672 for gsd with  112524 particles
gsd_to_G canvas has shape  (1000, 1000)
[[ 0 10]]
(0, 115, array([[ 0, 10],
       [ 1, 10],
       [ 2, 10],
       [ 3, 10],
       [ 4, 10],
       [ 5, 10],
       [ 6, 11]], dtype=int16))
Ran gsd_to_G in  3.8583879470825195 for a graph with  5704 nodes.
pre sub has  5704  nodes
post sub has  4589  nodes
post sub has  4589  nodes
4589
graph_shape is  [999, 999]  and img_shape is  (1018, 1536)
Loaded img in  0.0
img_bin has shape  (1000, 1000)
Added weights to a graph  with  4589 nodes in  0.334881067276001
4589
Graph has max  [1000 1000]
source coord is  [-10 500]
sink coord is  [1010  500]
Before connecting external nodes, G has vcount  4591
True  connected
After connecting external nodes, G has vcount  4591
(4591,) F
(4591, 4591) L
22.074339848222678


In [10]:
#Object oriented implementation test for continuous rotations
#Isotropic analysis
"""
From Wenbing: Sheet resistance is 83 Ω/sq in one direction and 92 Ω/sq in the other one
"""
from PIL import Image
import network
import cv2 as cv
import os

#os.remove('TestData/AgNWN_iso/Binarized/rot_skel.gsd')

g = network.ResistiveNetwork('TestData/AgNWN_iso')
g.binarize()

#thetas = np.linspace(0.001,350, 4)
thetas = (30,45,60,90)

#crop = 359

#Need to reassign weights from a temporary rotated binary image
#First write the temporary image
#Must calculate positions for the crop because this crop is origin cornered (and not origin centred, like the one above)
img_bin = cv.imread(g.stack_dir+'/slice0.tiff') #Original image
image_center = tuple(np.array(img_bin.shape[1::-1]) / 2)
short_length = img_bin.shape[img_bin.shape == max(img_bin.shape)]
long_length = max(img_bin.shape)
ISS = (short_length**2/2)**0.5
L1 = int((long_length - ISS)/2)
L3 = int(ISS+L1)
L2 = int((short_length - ISS)/2)
L4 = int(ISS+L2)
o_corn_crop = [L2,L1,L4,L3]
dims = L2-L1



R_j = 5
O_eff_df = []
Ax_df = []
Ay_df = []
theta_df = []
nodes = []
for theta in thetas:
    g.stack_to_gsd(name='rot_skel.gsd', crop=[500,1200,500,1200], rotate=theta)
    g.G_u()
    g.potential_distribution(0, [0,20], [700-20,700], R_j=R_j, rho_dim=0.659)
    base.Node_labelling(g, g.P, 'P', 'Binarized/Results.gsd')
    Ax, Ay = base.gyration_moments(g.Gr, sampling = 0.01)
    L = g.L
    Q = np.linalg.pinv(L)
    O_eff = Q[-1,-1]+Q[-2,-2]-2*Q[-1,-2]
    
    nodes.append(g.Gr.vcount())
    
    print(theta, O_eff, Ax, Ay)
    
    #stop
    
    
    O_eff_df.append(O_eff)
    Ax_df.append(Ax)
    Ay_df.append(Ay)
    theta_df.append(theta)

df_cont = pd.DataFrame(columns=['Theta','O_eff','Ax','Ay'])
df_cont['O_eff'] = O_eff_df
df_cont['Ax'] = Ax_df
df_cont['Ay'] = Ay_df
df_cont['Theta'] = theta_df
df_cont.to_csv('TestData/AgNWN_iso/AgNWN_iso.csv')

df_cont

[[  0   0   1]
 [  0   0   2]
 [  0   0  75]
 ...
 [  0 699 600]
 [  0 699 602]
 [  0 699 689]]
(700, 700)
Ran stack_to_gsd() in  0.2432701587677002 for gsd with  47442 particles
gsd_to_G canvas has shape  (700, 700)
[[0 2]]
(0, 15, array([[0, 2],
       [0, 1],
       [1, 0]], dtype=int16))
Ran gsd_to_G in  2.2408671379089355 for a graph with  3639 nodes.
pre sub has  3639  nodes
post sub has  3220  nodes
post sub has  3220  nodes
3220
graph_shape is  [699, 699]  and img_shape is  (1, 700, 700)
3220
Graph has max  [699, 699]
source coord is  [-10 349]
sink coord is  [709 349]
Before connecting external nodes, G has vcount  3222
True  connected
After connecting external nodes, G has vcount  3222
(3222,) F
(3222, 3222) L


implicit data copy when writing chunk: log/particles/P


30 82.30070990201877 54931.44237507795 468942.7115855379
[[  0   0   6]
 [  0   0   7]
 [  0   0   8]
 ...
 [  0 699 690]
 [  0 699 694]
 [  0 699 697]]
(700, 700)
Ran stack_to_gsd() in  0.37424302101135254 for gsd with  46753 particles
gsd_to_G canvas has shape  (700, 700)
[[0 8]]
(0, 42, array([[0, 8],
       [0, 7],
       [0, 6],
       [1, 5],
       [1, 4],
       [1, 3],
       [2, 2],
       [3, 1]], dtype=int16))
Ran gsd_to_G in  2.2129640579223633 for a graph with  3297 nodes.
pre sub has  3297  nodes
post sub has  2715  nodes
post sub has  2715  nodes
2715
graph_shape is  [699, 699]  and img_shape is  (1, 700, 700)
2715
Graph has max  [699, 699]
source coord is  [-10 349]
sink coord is  [709 349]
Before connecting external nodes, G has vcount  2717
True  connected
After connecting external nodes, G has vcount  2717
(2717,) F
(2717, 2717) L


implicit data copy when writing chunk: log/particles/P


45 86.07402584456256 17518.48735424869 126486.28313639548
[[  0   0  16]
 [  0   0  17]
 [  0   0  89]
 ...
 [  0 699 430]
 [  0 699 675]
 [  0 699 676]]
(700, 700)
Ran stack_to_gsd() in  0.35301828384399414 for gsd with  47607 particles
gsd_to_G canvas has shape  (700, 700)
[[ 0 16]]
(0, 78, array([[ 0, 16],
       [ 0, 17],
       [ 1, 18],
       [ 1, 19],
       [ 2, 20],
       [ 2, 21],
       [ 3, 22],
       [ 4, 22],
       [ 5, 22],
       [ 6, 22],
       [ 7, 22],
       [ 8, 23]], dtype=int16))
Ran gsd_to_G in  2.130038022994995 for a graph with  3727 nodes.
pre sub has  3727  nodes
post sub has  3145  nodes
post sub has  3145  nodes
3145
graph_shape is  [699, 699]  and img_shape is  (1, 700, 700)
3145
Graph has max  [699, 699]
source coord is  [-10 349]
sink coord is  [709 349]
Before connecting external nodes, G has vcount  3147
True  connected
After connecting external nodes, G has vcount  3147
(3147,) F
(3147, 3147) L


implicit data copy when writing chunk: log/particles/P


60 81.76266775667916 66887.82252451508 348926.8597992321
[[  0   0  31]
 [  0   0 182]
 [  0   0 281]
 ...
 [  0 699 697]
 [  0 699 698]
 [  0 699 699]]
(700, 700)
Ran stack_to_gsd() in  0.3054628372192383 for gsd with  45871 particles
gsd_to_G canvas has shape  (700, 700)
[[ 0 31]]
(0, 74, array([[ 0, 31],
       [ 1, 31],
       [ 2, 31],
       [ 3, 30],
       [ 4, 30],
       [ 5, 30],
       [ 6, 30],
       [ 7, 30],
       [ 8, 30]], dtype=int16))
Ran gsd_to_G in  2.3192577362060547 for a graph with  2358 nodes.
pre sub has  2358  nodes
post sub has  1339  nodes
post sub has  1339  nodes
1339
graph_shape is  [699, 699]  and img_shape is  (1, 700, 700)
1339
Graph has max  [699, 699]
source coord is  [-10 349]
sink coord is  [709 349]
Before connecting external nodes, G has vcount  1341
True  connected
After connecting external nodes, G has vcount  1341
(1341,) F
(1341, 1341) L


implicit data copy when writing chunk: log/particles/P


90 113.47408552940087 69867.69840896559 64076.15556804852


,Theta,O_eff,Ax,Ay
0,30,82.300710,54931.442375,468942.711586
1,45,86.074026,17518.487354,126486.283136
2,60,81.762668,66887.822525,348926.859799
3,90,113.474086,69867.698409,64076.155568


In [66]:
#Object oriented implementation test for continuous rotations
#Aligned analysis
stop
from PIL import Image
import network
import cv2 as cv

g = network.ResistiveNetwork('TestData/AgNWN_10um')
g.binarize()

#thetas = np.linspace(0.001,350, 4)
thetas = (30,45)

#crop = 359

#Need to reassign weights from a temporary rotated binary image
#First write the temporary image
#Must calculate positions for the crop because this crop is origin cornered (and not origin centred, like the one above)
img_bin = cv.imread(g.stack_dir+'/slice0.tiff') #Original image
image_center = tuple(np.array(img_bin.shape[1::-1]) / 2)
short_length = img_bin.shape[img_bin.shape == max(img_bin.shape)]
long_length = max(img_bin.shape)
ISS = (short_length**2/2)**0.5
L1 = int((long_length - ISS)/2)
L3 = int(ISS+L1)
L2 = int((short_length - ISS)/2)
L4 = int(ISS+L2)
o_corn_crop = [L2,L1,L4,L3]
dims = L2-L1



R_j = 0
O_eff_df = []
Ax_df = []
Ay_df = []
theta_df = []
nodes = []
for theta in thetas:
    g.stack_to_gsd(name='/Rotations/ObjectImplementation/rot_skel.gsd', crop=[L2,L4,L1,L3], rotate=theta)
    g.G_u()
    with Image.open(g.stack_dir+'/slice0.tiff') as im:
        img_crop = im.crop(box=[L2,L1,L4,L3])
        img_rot=im.rotate(theta*180/np.pi) #Rotate
        img_rot.save(g.dir+'/Rotations/ObjectImplementation/slice0.tiff') 

    g.potential_distribution(0, [0,20], [727-20,727], R_j=R_j, rho_dim=0.89)
    base.Node_labelling(g, g.P, 'P', 'test.gsd')
    Ax, Ay = base.gyration_moments(g.Gr, sampling = 0.01)
    L = g.L
    Q = np.linalg.pinv(L)
    O_eff = Q[-1,-1]+Q[-2,-2]-2*Q[-1,-2]
    
    nodes.append(g.Gr.vcount())
    
    print(theta, O_eff, Ax, Ay)
    
    #stop
    
    
    O_eff_df.append(O_eff)
    Ax_df.append(Ax)
    Ay_df.append(Ay)
    theta_df.append(theta)

df_cont = pd.DataFrame(columns=['Theta','O_eff','Ax','Ay'])
df_cont['O_eff'] = O_eff_df
df_cont['Ax'] = Ax_df
df_cont['Ay'] = Ay_df
df_cont['Theta'] = theta_df
#df_cont.to_csv('AgNWN.csv')

df_cont

Ran stack_to_gsd() in  0.1953139305114746 for gsd with  63237 particles
gsd_to_G canvas has shape  (719, 719)
[[ 0 87]]
(0, 88, array([[ 0, 87],
       [ 0, 86],
       [ 0, 85],
       [ 1, 84],
       [ 2, 83],
       [ 3, 82],
       [ 4, 82],
       [ 5, 81],
       [ 6, 81],
       [ 7, 80],
       [ 8, 79]], dtype=int16))
Ran gsd_to_G in  2.2781827449798584 for a graph with  6131 nodes.
pre sub has  6131  nodes
post sub has  5904  nodes
post sub has  5904  nodes
5904
graph_shape is  [718, 718]  and img_shape is  (1018, 1536)
Loaded img in  9.5367431640625e-07
img_bin has shape  (719, 719)
Added weights to a graph  with  5904 nodes in  0.5258090496063232
5904
Graph has max  [719 719]
source coord is  [-10 359]
sink coord is  [729 359]
Before connecting external nodes, G has vcount  5906
True  connected
After connecting external nodes, G has vcount  5906
(5906,) F
(5906, 5906) L


implicit data copy when writing chunk: log/particles/P


30 16.978278718184775 297484750.40563 2184246658.04216
Ran stack_to_gsd() in  0.21865010261535645 for gsd with  57507 particles
gsd_to_G canvas has shape  (719, 719)
[[0 1]]
(1, 51, array([[  0, 207],
       [  0, 206],
       [  0, 205],
       [  1, 204],
       [  1, 203],
       [  2, 202],
       [  2, 201],
       [  2, 200],
       [  3, 199],
       [  3, 198],
       [  3, 197],
       [  4, 196],
       [  4, 195],
       [  5, 194],
       [  6, 193]], dtype=int16))
Ran gsd_to_G in  2.136204957962036 for a graph with  4339 nodes.
pre sub has  4339  nodes
post sub has  4082  nodes
post sub has  4082  nodes
4082
graph_shape is  [718, 718]  and img_shape is  (1018, 1536)
Loaded img in  0.0
img_bin has shape  (719, 719)


/Users/alaink/Software/StructuralGTEdits/StructuralGT/GetWeights_3d.py:32: RuntimeWarning: invalid value encountered in true_divide
  return vec/np.linalg.norm(vec)


Added weights to a graph  with  4082 nodes in  0.40233540534973145
4082
Graph has max  [719 719]
source coord is  [-10 359]
sink coord is  [729 359]
Before connecting external nodes, G has vcount  4084
True  connected
After connecting external nodes, G has vcount  4084
(4084,) F
(4084, 4084) L


implicit data copy when writing chunk: log/particles/P


45 29.680789746264274 85748540.35406582 1456656168.023311


,Theta,O_eff,Ax,Ay
0,30,16.978279,2.974848e+08,2.184247e+09
1,45,29.680790,8.574854e+07,1.456656e+09


In [ ]:
#Statistical error
import numpy as np
import pandas as pd
import itertools
import base
import igraph as ig
import os
directory = 'TestData/AgNWN_Aligned/'
options = {"Thresh_method": 2, "gamma": 1.6,
           "md_filter": 0, "g_blur": 1, "autolvl": 0,
           "fg_color": 0, "laplacian": 0, "scharr": 0,
           "sobel": 0, "lowpass": 0, "asize": 11,
           "bsize": "3", "wsize": 10, "thresh": 127}

#Only need to run these once
if 1==2:
    for file in os.listdir(directory):
        try:
            base.ExpProcess(directory+file, options_dict=options)
            base.stack_to_gsd(directory+file+'/Binarized',directory+file+'/skel.gsd')
        except NotADirectoryError:
            pass
        except FileNotFoundError:
            pass
        except ImageDirectoryError:
            pass



df_group = pd.DataFrame(columns=['Axis','O_eff','Filename'])
axes_df = []
O_eff_df = []
Filename_df = []
axes = (0,1)
R_j=10
for file in os.listdir(directory):
    try:
        image_name = directory + file + '/Binarized'
        gsd_name  = directory+file+'/skel.gsd'
        for axis in axes:
            VC,G=base.voltage_distribution(image_name, gsd_name, axis, [0,20], [780,800], crop=[0,800,0,800,0,800], _2d=True, R_j=R_j, rho_dim=0.89)
            L = np.load(directory+file+'/Binarized/L.npy')
            Q = np.linalg.pinv(L)  
            O_eff = Q[-1,-1]+Q[-2,-2]-2*Q[-1,-2]
            axes_df.append(axis)
            O_eff_df.append(O_eff)
            Filename_df.append(file)
    except OSError:
        pass

df_group['O_eff'] = O_eff_df
df_group['Axis'] = axes_df
df_group['Filename'] = Filename_df

In [ ]:
#Continuous rotation analysis
#Testing base.py implementation
#This implementation rotates the image itself

directory = '/Users/alaink/Software/StructuralGTEdits/StructuralGT/TestData/AgNWN_10um'
import base
import numpy as np
from PIL import Image
import cv2 as cv

#thetas = np.linspace(0.01,350, 36)
thetas = (60, 120)

crop = 359

o_cent_crop=[-crop,crop,-crop,crop,-crop,crop]

#Need to reassign weights from a temporary rotated binary image
#First write the temporary image
#Must calculate positions for the crop because this crop is origin cornered (and not origin centred, like the one above)
img_bin = cv.imread(directory+'/Binarized/slice0.tiff') #Original image
image_center = tuple(np.array(img_bin.shape[1::-1]) / 2)
short_length = img_bin.shape[img_bin.shape == max(img_bin.shape)]
long_length = max(img_bin.shape)
ISS = (short_length**2/2)**0.5
L1 = (long_length - ISS)/2
L3 = ISS+L1
L2 = (short_length - ISS)/2
L4 = ISS+L2
o_corn_crop = (L2,L1,L4,L3)

R_j = 0
O_eff_df = []
Ax_df = []
Ay_df = []
theta_df = []
for theta in thetas:
    base.stack_to_gsd(directory + '/Binarized', directory + '/Rotations/rot_skel.gsd', crop=o_cent_crop, rotate=theta)
    with Image.open(directory+'/Binarized/slice0.tiff') as im:
        img_crop = im.crop(box=o_corn_crop)
        img_rot=im.rotate(theta*180/np.pi) #Rotate
        img_crop.save(directory+'/Rotations/slice0.tiff') 

    G = base.Node_labelling(base.voltage_distribution, 'V2', directory+'/Rotations/rot_skel.gsd', directory+'/Rotations', directory+'/Rotations/rot_skel.gsd', 0, [0,20], [crop-20,crop], _2d=True, R_j=0, rho_dim=0.89)
    print(G.vcount())
    Ax, Ay = base.gyration_moments(G, sampling = 0.15)
    L = np.load(directory+'/Rotations/L.npy')
    Q = np.linalg.pinv(L)
    O_eff = Q[-1,-1]+Q[-2,-2]-2*Q[-1,-2]
    
    O_eff_df.append(O_eff)
    Ax_df.append(Ax)
    Ay_df.append(Ay)
    theta_df.append(theta)

df_cont = pd.DataFrame(columns=['Theta','O_eff','Ax','Ay'])
df_cont['O_eff'] = O_eff_df
df_cont['Ax'] = Ax_df
df_cont['Ay'] = Ay_df
df_cont['Theta'] = theta_df
df_cont.to_csv('AgNWN.csv')

In [ ]:
df_cont = pd.DataFrame(columns=['Axis','thets','O_eff'])
df_cont['O_eff'] = O_eff_df
df_cont['Ax'] = Ax_df
df_cont['Ay'] = Ay_df
df_cont['Theta'] = theta_df

df_cont

In [ ]:
directory = 'TestData/AgNWN_Aligned/'
for file in os.listdir(directory):
    try:
        old_name = file + '/' + file + '.tiff'
        new_name = file + '/' + 'Z' + file + '.tiff'
        os.rename(directory+old_name, directory+new_name)
    except FileNotFoundError:
        pass
    except NotADirectoryError:
        pass

In [ ]:
import random
import numpy as np
arr=np.array([1,2,3,4,1,2,1,2,3,3])
mask = np.random.rand(10) > 0
mask

In [ ]:
arr[mask]

In [ ]:
#Test gyration moments estimations
import base
import random
import time
directory = '/Users/alaink/Software/StructuralGTEdits/StructuralGT/TestData/AgNWN_10um/Binarized'

axis=1
#VC,G=base.voltage_distribution(directory, directory+'/skel.gsd', axis, [0,20], [780,800], crop=[0,800,0,800,0,800], _2d=True, R_j=10, rho_dim=0.89)

def gyration_moments(G, sampling=1):
#Serial implementation
    Ax=0
    Ay=0
    node_count = np.asarray(list(range(G.vcount())))
    mask = np.random.rand(G.vcount()) > (1-sampling)
    trimmed_node_count = node_count[mask]
    for i in trimmed_node_count:
        for j in trimmed_node_count:
            if i >= j:    #Symetric matrix
                continue
            path = G.get_shortest_paths(i,to=j)
            Ax_term = 0
            Ay_term = 0
            for hop_s,hop_t in zip(path[0][0:-1],path[0][1::]):
                weight = G.es[G.get_eid(hop_s,hop_t)]['weight']
                Ax_term = Ax_term + ((weight*(int(G.vs[hop_s]['o'][0])-int(G.vs[hop_t]['o'][0])))**2)
                Ay_term = Ay_term + ((weight*(int(G.vs[hop_s]['o'][1])-int(G.vs[hop_t]['o'][1])))**2)
            Ax = Ax + (Ax_term)
            Ay = Ay + (Ay_term)

    return Ax, Ay


for i in range(5):
    start = time.time()
    Ax,Ay = gyration_moments(G,sampling=0.33)
    end = time.time()
    
    
    print(Ax/Ay, ' in ', end-start)

In [ ]:
import os
from PIL import Image
crop = [0,0,1536,1018]
for file in os.listdir('TestData/AgNWN_Aligned'):
    try:
        new_name = file.replace(',','').replace('(','').replace(')','')
        with Image.open('TestData/AgNWN_Aligned/'+new_name+'/'+new_name+'.tiff') as im:
            img_crop = im.crop(box=crop)
            img_crop.save('TestData/AgNWN_Aligned/'+new_name+'/'+'DIM'+new_name+'.tiff') 
        
    except NotADirectoryError:
        pass
    except FileNotFoundError:
        pass

In [ ]:
#Rotation matrix test
#Generally ineffective implementation (transforms positions after reading)
directory = '/Users/alaink/Software/StructuralGTEdits/StructuralGT/TestData/AgNWN_10um'
import gsd.hoomd
import matplotlib.pyplot as plt
from numpy import logical_and as a
from scipy.spatial.transform import Rotation as R
from PIL import Image
from skimage.morphology import binary_closing, ball, diamond, square, disk
#Crop to outer square
outer_crop = 1000
crop = [0,outer_crop,0,outer_crop,0,outer_crop]
frame = gsd.hoomd.open(name=directory+'/skel.gsd', mode='rb')[0]
p = base.shift(frame.particles.position.astype(int)).T
positions = p.T[a(a(a(a(a(p[0]>=crop[0],p[0]<=crop[1]),p[1]>=crop[2]),p[1]<=crop[3]),p[2]>=crop[4]),p[2]<=crop[5])]

#Centre on origin, rotate and recrop to the inner square
positions = base.oshift(positions)
theta = 0#np.pi/4
rot_mat = R.from_quat([np.cos(theta/2),0,0,np.sin(theta/2)]).as_matrix()
p = np.matmul(positions,rot_mat).T.astype(int)
inner_crop = int(outer_crop/(2*2**0.5))
crop = [0,inner_crop,0,inner_crop,0,inner_crop]
positions = base.shift(positions)
positions = p.T[a(a(a(a(a(p[0]>=crop[0],p[0]<=crop[1]),p[1]>=crop[2]),p[1]<=crop[3]),p[2]>=crop[4]),p[2]<=crop[5])]


#Fill rotated skel to account for edges broken during the rotation
dims = np.asarray(list(max(positions.T[i])+1 for i in (0,1,2)))
canvas = np.zeros(dims)
canvas[positions.T[0], positions.T[1], positions.T[2]] = 1
img_bin = canvas
img_bin = binary_closing(img_bin, selem=ball(3))
positions = np.asarray(np.where(img_bin != 0)).T


#Write rotated positions to new temporary gsd
s = gsd.hoomd.Snapshot()
s.particles.N = len(positions)
s.particles.types = ['A']
s.particles.typeid = ['0']*s.particles.N
s.particles.position = positions
print(positions)



with gsd.hoomd.open(name=directory+'/Rotations/rot_skel.gsd', mode='wb') as f:
    f.append(s)

#Need to reassign weights from a temporary rotated binary image
#First write the temporary image
with Image.open(directory+'/Binarized/slice0.tiff') as im:
    #img_crop=im.crop((0,0,outer_crop, outer_crop)) #Crop to outer square
    img_rot=im.rotate(theta*180/np.pi) #Rotate
    
    #Calculate inner square positions and crop to it
    bottom = (2*outer_crop*2**0.5 - outer_crop)/(4*2**0.5)
    top = outer_crop - bottom
    img_crop = img_rot.crop((591,686,945,332))
    img_crop.save(directory+'/Rotations/slice0.tiff') 

VC,G=base.voltage_distribution(directory+'/Rotations', directory+'/Rotations/rot_skel.gsd', 0, [0,20], [inner_crop-20,inner_crop], crop=crop, _2d=True, R_j=R_j, rho_dim=1.62)
L = np.load(directory+'/Rotations/L.npy')
Q = np.linalg.pinv(L)
O_eff = Q[-1,-1]+Q[-2,-2]-2*Q[-1,-2]

fig = plt.figure()
ax = fig.add_subplot(111)

plt.scatter(positions.T[1],positions.T[2],s=0.005)

ax.set_aspect('equal', adjustable='box')

In [ ]:
G.vcount()